In [2]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>

sweeps = [
    'y3ya5bpc', 'qb5zkdsx', 'l72dh1w7', 'kuvylfn5', 'e6rkfmy6', 'ucanqlze',
    'ry0ss5vc', 'leziylwb', 'e63yxyoh', 'n13bozse', '1ybj6xzh', 'jk2b8bo6',
    'pawx9wi3', 'kxl2l1qj', '78xmz3k9', 'eizv5hgc', 'a67196fk', '9pyz272w'
]

summary_list, config_list, type_list, val_loss_list = [], [], [], []
for sweep in sweeps:
    try:
        s = api.sweep(f"read/read/sweeps/{sweep}")
    except:
        continue
    for run in s.runs:
        his = run.history()
        if 'CIDEr' in his:
            his = his[~his['CIDEr'].isna()]
            cider = max(his['CIDEr'].max(), run.summary['CIDEr'])
            val_loss = his['val_loss'].min()
            lr = run.config['learning_rate']

            summary_list.append(cider)
            config_list.append(lr)
            type_list.append(s.name)
            val_loss_list.append(val_loss)

runs_df = pd.DataFrame({
    "CIDEr": summary_list,
    "lr": config_list,
    "sweep_id": type_list,
    "val_loss": val_loss_list
})

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


In [3]:
runs_df

,CIDEr,lr,sweep_id,val_loss
0,0.000000e+00,0.000353,Full Decoder Only,10.273881
1,0.000000e+00,0.000320,Full Decoder Only,10.839893
2,7.166642e-05,0.000999,Full Decoder Only,6.952788
3,1.328771e-08,0.000525,Full Decoder Only,8.221014
4,0.000000e+00,0.000470,Full Decoder Only,9.219606
...,...,...,...,...
170,6.292992e-01,0.000993,rnn 512,1.442932
171,6.591082e-01,0.000854,rnn 512,1.443572
172,6.985016e-01,0.000993,rnn 512,1.494614
173,8.608575e-01,0.000527,rnn 256 bert large,0.960141


In [4]:
runs_df.to_csv("runs.csv", index=False)

In [ ]:
pd.DataFrame([
    group.sort_values("CIDEr", ascending=False).iloc[0].to_dict()
    for _, group in runs_df.groupby("ft_type")
])


In [5]:
!unset LD_LIBRARY_PATH

In [6]:
from coco import *
from adapter_utils import *
from utils import *
from transformers.adapters import LoRAConfig, PrefixTuningConfig


data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=16,
    val_batch_size=128)

peft_confg = RNNAdapterConf(outputs_scaling_factor=0.8551,
                            input_dim=1024,
                            rnn_dim=256,
                            num_layers=1,
                            rnn_type="gru",
                            bidirectional=False,
                            layer_norm_before=True)

model = ImageCaptioningModel(
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    text_model_name_or_path="bert-large-uncased",
    data_module=data_module,
    learning_rate=0.0008540106397 * 0.70710678,
    peft_conf=peft_confg,
)

trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    callbacks=[
        EarlyStopping(monitor="val_loss",
                        min_delta=0.00,
                        patience=3,
                        verbose=True),
        TQDMProgressBar(),
    ],
    default_root_dir="/data/home/ngjhn/read",
    num_sanity_val_steps=0,
    check_val_every_n_epoch=1,
)

trainer.fit(model, data_module)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['bert.encoder.layer.5.crossattention.self.value.bias', 'bert.encoder.layer.12.crossattention.output.dense.weight', 'bert.encoder.layer.22.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.e

Enables rnn-adapter tuning; num trainable params 2,234,880, ratio 0.005
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | VisionEncoderDecoderModel | 525 M 
1 | val_loss  | AvgMetric                 | 0     
2 | test_loss | AvgMetric                 | 0     
--------------------------------------------------------
2.2 M     Trainable params
523 M     Non-trainable params
525 M     Total params
2,101.587 Total estimated model params size (MB)


...

Epoch 7: 100%|██████████| 5174/5174 [34:41<00:00,  2.49it/s, v_num=46, loss=1.610]


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(








































100%|██████████| 40/40 [00:57<00:00,  1.44s/it]


loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 307085 tokens at 1325905.07 tokens per second.
PTBTokenizer tokenized 58838 tokens at 462994.02 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 48640, 'reflen': 48052, 'guess': [48640, 43640, 38640, 33640], 'correct': [33040, 17182, 7952, 3633]}
ratio: 1.012236743527824
Bleu_1: 0.679
Bleu_2: 0.517
Bleu_3: 0.380
Bleu_4: 0.278
computing METEOR score...


In [ ]:
model = model.load_from_checkpoint(
    "/data/home/ngjhn/read/lightning_logs/rnn_adapter/name=0-lr:config.learning_rate=0.0000-epoch=39-val_loss=1.03.ckpt",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    text_model_name_or_path="bert-large-uncased",
    data_module=data_module,
    learning_rate=0.0008540106397,
    peft_conf=peft_confg,
)


In [ ]:
model.evaluate("cuda:0")

# RuntimeError: input.size(-1) must be equal to input_size. Expected 768, got 1024

In [ ]:
!nvidia-smi